In [463]:
import os
import numpy as np
from pandas import read_csv
from collections import Counter
import pandas as pd

In [13]:
exp_param_directory = '../experimental_parameters/'

In [14]:
exp_param_file = exp_param_directory + 'level_matching_parameters.csv'

In [573]:
test_greebles_file = exp_param_directory + 'test_greeble_images.csv'
greebles = read_csv(test_greebles_file).values
greebles = [g[0] for g in greebles]
greebles

['m1~110-v1.tif',
 'm1~16-v1.tif',
 'm4~49-v1.tif',
 'f3~39-v1.tif',
 'f2~23-v1.tif',
 'f1~14-v1.tif',
 'm1~11-v1.tif',
 'f4~41-v1.tif',
 'm2~210-v1.tif',
 'f3~38-v1.tif',
 'f3~310-v1.tif',
 'm4~46-v1.tif',
 'm2~22-v1.tif',
 'f4~44-v1.tif',
 'f3~34-v1.tif',
 'f4~42-v1.tif',
 'm3~31-v1.tif',
 'f3~32-v1.tif',
 'm1~13-v1.tif',
 'f5~53-v1.tif',
 'f2~24-v1.tif',
 'm2~23-v1.tif',
 'm1~14-v1.tif',
 'm5~56-v1.tif',
 'm4~45-v1.tif',
 'f5~55-v1.tif',
 'f5~57-v1.tif',
 'm5~55-v1.tif',
 'f1~15-v1.tif',
 'm4~44-v1.tif',
 'm3~33-v1.tif',
 'm5~54-v1.tif',
 'f1~13-v1.tif',
 'f5~52-v1.tif',
 'f5~510-v1.tif',
 'm2~21-v1.tif',
 'm2~25-v1.tif',
 'f2~25-v1.tif',
 'm3~32-v1.tif',
 'm4~41-v1.tif']

In [52]:
n = len(greebles)
num_match = 4

In [49]:
greebles = [g[0]+g.split('~')[1].split('-')[0] for g in greebles]

In [295]:
male = [g for g in greebles if g[0] == 'm']
female = [g for g in greebles if g[0] == 'f']
f1 = [g for g in greebles if g[1] == '1']
f2 = [g for g in greebles if g[1] == '2']
f3 = [g for g in greebles if g[1] == '3']
f4 = [g for g in greebles if g[1] == '4']
f5 = [g for g in greebles if g[1] == '5']
families = [f1, f2, f3, f4, f5]

In [296]:
levels = ['family', 'individual', 'sex']

In [356]:
def update(counter, selected):
    cp = counter.copy()
    for s in selected:
        cp[s] += 1
    return cp

In [498]:
def compute_leftover():
    leftover = list()
    for k in gcounter.keys():
        if gcounter[k]<2:
            leftover.append(k)
    for g in greebles:
        if g not in gcounter.keys():
            leftover.append(g)
    return leftover

# individual

In [358]:
def individual_sampling(counter, num, sample_list):
    g_ind = np.random.choice(len(sample_list), num_match)
    rep_ind = g_ind[np.random.choice(num_match,1)]
    trial_idx = np.append(rep_ind, g_ind)
    gbs = np.array(sample_list)[trial_idx]
    return gbs

In [359]:
def generate_individual_trials(counter, num_match, sample_list):
    gbs = individual_sampling(counter, num_match, sample_list)
#     print(gbs)
    counter_copy = update(gcounter, gbs)
    if counter_copy.most_common(1)[0][1]>2:
        return generate_individual_trials(counter, num_match, sample_list)
    else:
        return gbs, counter_copy

# family

In [360]:
def family_sampling(counter, num):
    norepeat = True
    while norepeat:
        g_ind = np.random.choice(len(greebles), num_match+1)
        gs = np.array(greebles)[g_ind]
        fs = [g[1]for g in gs]
        if len(list(set(fs))) < len(fs):
            norepeat = False
    return np.array(greebles)[g_ind]

In [361]:
def generate_family_trials(counter, num_match):
    gbs = family_sampling(counter, num_match)
    counter_copy = update(gcounter, gbs)
    if counter_copy.most_common(1)[0][1]>2:
        return generate_family_trials(counter, num_match)
    else:
        return gbs, counter_copy

# sex

In [435]:
def sex_sampling(counter, num, sample_list):
    bad_ratio = True
    while bad_ratio:
        g_ind = np.random.choice(len(sample_list), num_match+1)
        gs = np.array(sample_list)[g_ind]
        f = [1 for g in gs if g[0] == 'f']
        m = [1 for g in gs if g[0] == 'm']
        if abs(sum(f) - sum(m)) < 4:
            bad_ratio = False
    return np.array(sample_list)[g_ind]
def generate_sex_trials(counter, num_match, sample_list):
    gbs = sex_sampling(counter, num_match, sample_list)
    counter_copy = update(gcounter, gbs)
    if counter_copy.most_common(1)[0][1]>2:
        return generate_sex_trials(counter, num_match, sample_list)
    else:
        return gbs, counter_copy

In [490]:
gcounter = Counter()
individual_samples = list()
for i in range(2): #individuals
    gbs, gcounter = generate_individual_trials(gcounter, num_match, female)
    individual_samples.append(gbs)
for i in range(2): #individuals
    gbs, gcounter = generate_individual_trials(gcounter, num_match, male)
    individual_samples.append(gbs)
individual_samples

[array(['f38', 'f25', 'f39', 'f57', 'f38'], dtype='|S4'),
 array(['f13', 'f32', 'f44', 'f39', 'f13'], dtype='|S4'),
 array(['m23', 'm14', 'm56', 'm23', 'm210'], dtype='|S4'),
 array(['m32', 'm22', 'm21', 'm32', 'm11'], dtype='|S4')]

In [491]:
family_samples = list()

In [ ]:
for i in range(6):
    gbs, gcounter = generate_family_trials(gcounter, num_match)
    family_samples.append(gbs)

In [499]:
family_samples

[array(['m110', 'm33', 'f41', 'f34', 'm21'], dtype='|S4'),
 array(['f14', 'm13', 'm14', 'f34', 'm44'], dtype='|S4'),
 array(['m56', 'f57', 'm210', 'm11', 'f41'], dtype='|S4'),
 array(['f25', 'm33', 'f14', 'f32', 'm25'], dtype='|S4'),
 array(['f510', 'm22', 'm41', 'm16', 'm25'], dtype='|S4'),
 array(['m45', 'm54', 'm16', 'f53', 'm49'], dtype='|S4')]

In [500]:
sex_samples = list()

In [512]:
leftover = compute_leftover()
gbs, gcounter = generate_sex_trials(gcounter, num_match, leftover)
sex_samples.append(gbs)

In [513]:
sex_samples

[array(['f53', 'f23', 'm49', 'f42', 'm110'], dtype='|S4'),
 array(['f42', 'f44', 'f24', 'f24', 'm45'], dtype='|S4'),
 array(['f52', 'm44', 'm31', 'f23', 'm41'], dtype='|S4'),
 array(['m46', 'm55', 'm46', 'f310', 'f52'], dtype='|S4'),
 array(['m13', 'm54', 'f510', 'm31', 'f55'], dtype='|S4'),
 array(['m55', 'f55', 'f15', 'f310', 'f15'], dtype='|S4')]

In [517]:
#correct some orders of family presentation
family_samples[0] = np.array(['m33', 'm110', 'f41', 'f34', 'm21'])
family_samples[3] = np.array(['m33', 'f25', 'f14', 'f32', 'm25'])
family_samples[4] = np.array(['m22', 'f510', 'm41', 'm16', 'm25'])
family_samples

[array(['m33', 'm110', 'f41', 'f34', 'm21'], dtype='|S4'),
 array(['f14', 'm13', 'm14', 'f34', 'm44'], dtype='|S4'),
 array(['m56', 'f57', 'm210', 'm11', 'f41'], dtype='|S4'),
 array(['m33', 'f25', 'f14', 'f32', 'm25'], dtype='|S3'),
 array(['m22', 'f510', 'm41', 'm16', 'm25'], dtype='|S4'),
 array(['m45', 'm54', 'm16', 'f53', 'm49'], dtype='|S4')]

In [522]:
#correct some orders of set presentation
sex_samples[5] = np.array(['f55','f15', 'f310', 'f15', 'm55'])

# Save sampling procedure

In [546]:
trials = range(16)
np.random.shuffle(trials)
trials

[10, 1, 3, 0, 6, 13, 9, 4, 11, 12, 7, 2, 14, 5, 8, 15]

In [575]:
def gname(g):
    s = g[0] + g[1] + '~' + g[1:] + '-v1.tif'
    return s

In [576]:
df = pd.DataFrame()
t = 0
for g in individual_samples:
    vd = dict()
    vd['level'] = 'individual'
    vd['sample'] = gname(g[0])
    vd['match1'] = gname(g[1])
    vd['match2'] = gname(g[2])
    vd['match3'] = gname(g[3])
    vd['match4'] = gname(g[4])
    vd['trial'] = int(trial[t])
    df = df.append(vd, ignore_index=True)
    t+=1
for g in family_samples:
    vd = dict()
    vd['level'] = 'family'
    vd['sample'] = gname(g[0])
    vd['match1'] = gname(g[1])
    vd['match2'] = gname(g[2])
    vd['match3'] = gname(g[3])
    vd['match4'] = gname(g[4])
    vd['trial'] = int(trial[t])
    df = df.append(vd, ignore_index=True)
    t+=1
for g in sex_samples:
    vd = dict()
    vd['level'] = 'sex'
    vd['sample'] = gname(g[0])
    vd['match1'] = gname(g[1])
    vd['match2'] = gname(g[2])
    vd['match3'] = gname(g[3])
    vd['match4'] = gname(g[4])
    vd['trial'] = int(trial[t])
    df = df.append(vd, ignore_index=True)
    t+=1

In [577]:
print(df)

         level         match1         match2         match3         match4  \
0   individual   f2~25-v1.tif   f3~39-v1.tif   f5~57-v1.tif   f3~38-v1.tif   
1   individual   f3~32-v1.tif   f4~44-v1.tif   f3~39-v1.tif   f1~13-v1.tif   
2   individual   m1~14-v1.tif   m5~56-v1.tif   m2~23-v1.tif  m2~210-v1.tif   
3   individual   m2~22-v1.tif   m2~21-v1.tif   m3~32-v1.tif   m1~11-v1.tif   
4       family  m1~110-v1.tif   f4~41-v1.tif   f3~34-v1.tif   m2~21-v1.tif   
5       family   m1~13-v1.tif   m1~14-v1.tif   f3~34-v1.tif   m4~44-v1.tif   
6       family   f5~57-v1.tif  m2~210-v1.tif   m1~11-v1.tif   f4~41-v1.tif   
7       family   f2~25-v1.tif   f1~14-v1.tif   f3~32-v1.tif   m2~25-v1.tif   
8       family  f5~510-v1.tif   m4~41-v1.tif   m1~16-v1.tif   m2~25-v1.tif   
9       family   m5~54-v1.tif   m1~16-v1.tif   f5~53-v1.tif   m4~49-v1.tif   
10         sex   f2~23-v1.tif   m4~49-v1.tif   f4~42-v1.tif  m1~110-v1.tif   
11         sex   f4~44-v1.tif   f2~24-v1.tif   f2~24-v1.tif   m4

In [578]:
df = df.sort_values(by=['trial'])

In [579]:
df

,level,match1,match2,match3,match4,sample,trial
11,sex,f4~44-v1.tif,f2~24-v1.tif,f2~24-v1.tif,m4~45-v1.tif,f4~42-v1.tif,0.0
8,family,f5~510-v1.tif,m4~41-v1.tif,m1~16-v1.tif,m2~25-v1.tif,m2~22-v1.tif,1.0
3,individual,m2~22-v1.tif,m2~21-v1.tif,m3~32-v1.tif,m1~11-v1.tif,m3~32-v1.tif,2.0
5,family,m1~13-v1.tif,m1~14-v1.tif,f3~34-v1.tif,m4~44-v1.tif,f1~14-v1.tif,3.0
0,individual,f2~25-v1.tif,f3~39-v1.tif,f5~57-v1.tif,f3~38-v1.tif,f3~38-v1.tif,4.0
7,family,f2~25-v1.tif,f1~14-v1.tif,f3~32-v1.tif,m2~25-v1.tif,m3~33-v1.tif,5.0
4,family,m1~110-v1.tif,f4~41-v1.tif,f3~34-v1.tif,m2~21-v1.tif,m3~33-v1.tif,6.0
13,sex,m5~55-v1.tif,m4~46-v1.tif,f3~310-v1.tif,f5~52-v1.tif,m4~46-v1.tif,7.0
1,individual,f3~32-v1.tif,f4~44-v1.tif,f3~39-v1.tif,f1~13-v1.tif,f1~13-v1.tif,8.0
6,family,f5~57-v1.tif,m2~210-v1.tif,m1~11-v1.tif,f4~41-v1.tif,m5~56-v1.tif,9.0


In [580]:
df.to_csv(exp_param_file)

In [582]:
exp_param = read_csv(exp_param_file, header=0)

In [583]:
kk = exp_param.values

In [584]:
kk

array([[11, 'sex', 'f4~44-v1.tif', 'f2~24-v1.tif', 'f2~24-v1.tif',
        'm4~45-v1.tif', 'f4~42-v1.tif', 0.0],
       [8, 'family', 'f5~510-v1.tif', 'm4~41-v1.tif', 'm1~16-v1.tif',
        'm2~25-v1.tif', 'm2~22-v1.tif', 1.0],
       [3, 'individual', 'm2~22-v1.tif', 'm2~21-v1.tif', 'm3~32-v1.tif',
        'm1~11-v1.tif', 'm3~32-v1.tif', 2.0],
       [5, 'family', 'm1~13-v1.tif', 'm1~14-v1.tif', 'f3~34-v1.tif',
        'm4~44-v1.tif', 'f1~14-v1.tif', 3.0],
       [0, 'individual', 'f2~25-v1.tif', 'f3~39-v1.tif', 'f5~57-v1.tif',
        'f3~38-v1.tif', 'f3~38-v1.tif', 4.0],
       [7, 'family', 'f2~25-v1.tif', 'f1~14-v1.tif', 'f3~32-v1.tif',
        'm2~25-v1.tif', 'm3~33-v1.tif', 5.0],
       [4, 'family', 'm1~110-v1.tif', 'f4~41-v1.tif', 'f3~34-v1.tif',
        'm2~21-v1.tif', 'm3~33-v1.tif', 6.0],
       [13, 'sex', 'm5~55-v1.tif', 'm4~46-v1.tif', 'f3~310-v1.tif',
        'f5~52-v1.tif', 'm4~46-v1.tif', 7.0],
       [1, 'individual', 'f3~32-v1.tif', 'f4~44-v1.tif', 'f3~39-v1.tif',
